## Final Project Submission

Please fill out:
* Student name: 
* Student pace: self paced / part time / full time
* Scheduled project review date/time: 
* Instructor name: 
* Blog post URL:


In [8]:
import pandas as pd
import sqlite3

#Let's open the csv file for movie gross earnings
df = pd.read_csv("bom.movie_gross.csv")

In [15]:
#Let's look at a few pieces of information
#Let's start by finding the highest domestic grossing movie
max_dom_gross = df['domestic_gross'].max()
print(df.loc[df['domestic_gross'] == max_dom_gross])

                             title studio  domestic_gross foreign_gross  year
1872  Star Wars: The Force Awakens     BV     936700000.0    1131561399  2015


In [18]:
df.loc[1872, 'foreign_gross'] = 1131561399 #Correcting the foreign gross amount to be accurate
df.loc[1873, 'foreign_gross'] = 1018130012
df.loc[3079, 'foreign_gross'] = 1369544272
df.loc[1874, 'foreign_gross'] = 1162040651
df.loc[2760, 'foreign_gross'] =  1009996733

In [33]:
#Let's find the top grossing movies that made at least 400000000 in the domestic box office.

df_range = df.loc[(df['domestic_gross'] > 400000000.0) & (df['domestic_gross'] <= 936700000.0)]
df_range

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
727,Marvel's The Avengers,BV,623400000.0,895500000,2012
729,The Dark Knight Rises,WB,448100000.0,636800000,2012
735,The Hunger Games,LGF,408000000.0,286400000,2012
1127,Frozen,BV,400700000.0,875700000,2013
1128,Iron Man 3,BV,409000000.0,805800000,2013
1131,The Hunger Games: Catching Fire,LGF,424700000.0,440300000,2013
1872,Star Wars: The Force Awakens,BV,936700000.0,1131561399,2015
1873,Jurassic World,Uni.,652300000.0,1018130012,2015
1875,Avengers: Age of Ultron,BV,459000000.0,946400000,2015


In [17]:
#let's look at the highest foreign grossing movie.
# But before that, we need to manipulate some data
#Let's look at the data types for the DF
df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 2037 entries, 0 to 3353
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           2037 non-null   object 
 1   studio          2033 non-null   object 
 2   domestic_gross  2009 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            2037 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 175.5+ KB


In [17]:
#Under foreign_gross, it appears that there are some null values, and that the type of the column is string.
#We need to remove null values, and convert to integer to find the highest foreign grossing movie.

df.dropna(subset = ['foreign_gross'], inplace = True)
df['foreign_gross'] = df['foreign_gross'].astype(str).astype(int)

In [26]:
#Finding highest foreign grossing movie
max_for_gross = df['foreign_gross'].max()
print(df.loc[df['foreign_gross'] == max_for_gross])

                       title studio  domestic_gross  foreign_gross  year
3079  Avengers: Infinity War     BV     678800000.0     1369544272  2018


Looking at the years for this data, it looks like the years range from 2010 to 2018. So between the years of 2010 and 2018, the top domestic grossing movie was "Star Wars: The Force Awakens" and the top foreign grossing movie was "Avengers: Infinity War". But what about for each year?

In [31]:
#Let's split up the years
df_2010 = df.loc[df['year'] == 2010]
df_2010.head()

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010


In [83]:
df_2011 = df.loc[df['year'] == 2011]
df_2011.head()

,title,studio,domestic_gross,foreign_gross,year
328,Harry Potter and the Deathly Hallows Part 2,WB,381000000.0,960500000,2011
329,Transformers: Dark of the Moon,P/DW,352400000.0,771400000,2011
330,Pirates of the Caribbean: On Stranger Tides,BV,241100000.0,804600000,2011
331,The Twilight Saga: Breaking Dawn Part 1,Sum.,281300000.0,430900000,2011
332,Mission: Impossible - Ghost Protocol,Par.,209400000.0,485300000,2011


In [82]:
df_2012 = df.loc[df['year'] == 2012]
df_2012.head()

,title,studio,domestic_gross,foreign_gross,year
727,Marvel's The Avengers,BV,623400000.0,895500000,2012
728,Skyfall,Sony,304400000.0,804200000,2012
729,The Dark Knight Rises,WB,448100000.0,636800000,2012
730,The Hobbit: An Unexpected Journey,WB (NL),303000000.0,718100000,2012
731,Ice Age: Continental Drift,Fox,161300000.0,715900000,2012


In [81]:
df_2013 = df.loc[df['year'] == 2013]
df_2013.head()

,title,studio,domestic_gross,foreign_gross,year
1127,Frozen,BV,400700000.0,875700000,2013
1128,Iron Man 3,BV,409000000.0,805800000,2013
1129,Despicable Me 2,Uni.,368100000.0,602700000,2013
1130,The Hobbit: The Desolation of Smaug,WB (NL),258399999.0,700000000,2013
1131,The Hunger Games: Catching Fire,LGF,424700000.0,440300000,2013


In [80]:
df_2014 = df.loc[df['year'] == 2014]
df_2014.head()

,title,studio,domestic_gross,foreign_gross,year
1477,Transformers: Age of Extinction,Par.,245400000.0,858600000,2014
1478,The Hobbit: The Battle of the Five Armies,WB (NL),255100000.0,700900000,2014
1479,Guardians of the Galaxy,BV,333200000.0,440200000,2014
1480,Maleficent,BV,241400000.0,517100000,2014
1481,The Hunger Games: Mockingjay - Part 1,LGF,337100000.0,418200000,2014


In [14]:
df_2015 = df.loc[df['year'] == 2015]
df_2015.head()

,title,studio,domestic_gross,foreign_gross,year
1872,Star Wars: The Force Awakens,BV,936700000.0,1131561399,2015
1873,Jurassic World,Uni.,652300000.0,1018130012,2015
1874,Furious 7,Uni.,353000000.0,1162040651,2015
1875,Avengers: Age of Ultron,BV,459000000.0,946400000,2015
1876,Minions,Uni.,336000000.0,823400000,2015


In [78]:
df_2016 = df.loc[df['year'] == 2016]
df_2016.head()

,title,studio,domestic_gross,foreign_gross,year
2322,Captain America: Civil War,BV,408100000.0,745200000,2016
2323,Rogue One: A Star Wars Story,BV,532200000.0,523900000,2016
2324,Finding Dory,BV,486300000.0,542300000,2016
2325,Zootopia,BV,341300000.0,682500000,2016
2326,The Jungle Book (2016),BV,364000000.0,602500000,2016


In [16]:
df_2017 = df.loc[df['year'] == 2017]
df_2017.head()

,title,studio,domestic_gross,foreign_gross,year
2758,Star Wars: The Last Jedi,BV,620200000.0,712400000,2017
2759,Beauty and the Beast (2017),BV,504000000.0,759500000,2017
2760,The Fate of the Furious,Uni.,226000000.0,1009996733,2017
2761,Despicable Me 3,Uni.,264600000.0,770200000,2017
2762,Jumanji: Welcome to the Jungle,Sony,404500000.0,557600000,2017


In [29]:
df_2018 = df.loc[df['year'] == 2018]
df_2018.head()

,title,studio,domestic_gross,foreign_gross,year
3079,Avengers: Infinity War,BV,678800000.0,1369544272,2018
3080,Black Panther,BV,700100000.0,646900000,2018
3081,Jurassic World: Fallen Kingdom,Uni.,417700000.0,891800000,2018
3082,Incredibles 2,BV,608600000.0,634200000,2018
3083,Aquaman,WB,335100000.0,812700000,2018


In [74]:
df_budget = pd.read_csv('tn.movie_budgets.csv')

#Let's get rid of the dollar signs and commas in the column values
df_budget['production_budget'] = df_budget['production_budget'].str.replace('$','')
df_budget['production_budget'] = df_budget['production_budget'].str.replace(',','')
df_budget['domestic_gross'] = df_budget['domestic_gross'].str.replace('$','')
df_budget['domestic_gross'] = df_budget['domestic_gross'].str.replace(',','')
df_budget['worldwide_gross'] = df_budget['worldwide_gross'].str.replace('$','')
df_budget['worldwide_gross'] = df_budget['worldwide_gross'].str.replace(',','')


#Now let's convert the strings to ints
df_budget['production_budget'] = df_budget['production_budget'].astype(str).astype(int)
df_budget['domestic_gross'] = df_budget['domestic_gross'].astype(str).astype(int)
df_budget['worldwide_gross'] = df_budget['domestic_gross'].astype(str).astype(int)
df_budget.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,425000000,760507625,760507625
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000,241063875,241063875
2,3,"Jun 7, 2019",Dark Phoenix,350000000,42762350,42762350
3,4,"May 1, 2015",Avengers: Age of Ultron,330600000,459005868,459005868
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000,620181382,620181382


In [87]:
max_budget = df_budget['production_budget'].max()
print(max_budget)

425000000


In [17]:
conn = sqlite3.connect("im.db")
pd.read_sql("""
        SELECT *
        FROM movie_basics
        JOIN movie_ratings
        USING(movie_id)
        JOIN directors
        USING(movie_id)
        JOIN persons
        USING(person_id)
        ORDER BY numvotes DESC
        LIMIT 20;
        """,conn
)

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes,person_id,primary_name,birth_year,death_year,primary_profession
0,tt1375666,Inception,Inception,2010,148.0,"Action,Adventure,Sci-Fi",8.8,1841066,nm0634240,Christopher Nolan,1970.0,None,"writer,producer,director"
1,tt1345836,The Dark Knight Rises,The Dark Knight Rises,2012,164.0,"Action,Thriller",8.4,1387769,nm0634240,Christopher Nolan,1970.0,None,"writer,producer,director"
2,tt1345836,The Dark Knight Rises,The Dark Knight Rises,2012,164.0,"Action,Thriller",8.4,1387769,nm0634240,Christopher Nolan,1970.0,None,"writer,producer,director"
3,tt1345836,The Dark Knight Rises,The Dark Knight Rises,2012,164.0,"Action,Thriller",8.4,1387769,nm0634240,Christopher Nolan,1970.0,None,"writer,producer,director"
4,tt1345836,The Dark Knight Rises,The Dark Knight Rises,2012,164.0,"Action,Thriller",8.4,1387769,nm0634240,Christopher Nolan,1970.0,None,"writer,producer,director"
5,tt0816692,Interstellar,Interstellar,2014,169.0,"Adventure,Drama,Sci-Fi",8.6,1299334,nm0634240,Christopher Nolan,1970.0,None,"writer,producer,director"
6,tt0816692,Interstellar,Interstellar,2014,169.0,"Adventure,Drama,Sci-Fi",8.6,1299334,nm0634240,Christopher Nolan,1970.0,None,"writer,producer,director"
7,tt1853728,Django Unchained,Django Unchained,2012,165.0,"Drama,Western",8.4,1211405,nm0000233,Quentin Tarantino,1963.0,None,"writer,actor,producer"
8,tt0848228,The Avengers,The Avengers,2012,143.0,"Action,Adventure,Sci-Fi",8.1,1183655,nm0923736,Joss Whedon,1964.0,None,"writer,producer,director"
9,tt0848228,The Avengers,The Avengers,2012,143.0,"Action,Adventure,Sci-Fi",8.1,1183655,nm0923736,Joss Whedon,1964.0,None,"writer,producer,director"
